Practical Data Science
Project 2

Josh Ragen
Jeremy Xue

Permalink to full data source
http://ghdx.healthdata.org/gbd-results-tool?params=gbd-api-2016-permalink/c992f36d933723fd033ccdb9c3ef2a33

Permalink to specific data source (just HIV/AIDS, TB, Malaria)
http://ghdx.healthdata.org/gbd-results-tool?params=gbd-api-2016-permalink/acffd0c7209d119075df13f7fb5802da


Loading the data in:

In [207]:
import pandas as pd
import numpy as np
import copy

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import scipy.stats as stats
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

ImportError: cannot import name 'CategoricalEncoder'

In [3]:
#dat = pd.read_csv('full_data.csv')
dat = pd.read_csv('reduced_data.csv')
dat = dat.drop(['upper', 'lower', 'measure', 'sex'], axis = 1)
dat

,location,age,cause,metric,year,val
0,Kenya,Under 5,Tuberculosis,Number,2000,333.036316
1,Kenya,Under 5,Tuberculosis,Rate,2000,6.454670
2,Kenya,Under 5,Neglected tropical diseases and malaria,Number,2000,9546.056132
3,Kenya,Under 5,Neglected tropical diseases and malaria,Rate,2000,185.014795
4,Kenya,5-14 years,Neglected tropical diseases and malaria,Number,2000,1300.215171
5,Kenya,5-14 years,Neglected tropical diseases and malaria,Rate,2000,14.941310
6,Kenya,15-49 years,Neglected tropical diseases and malaria,Number,2000,1533.580658
7,Kenya,15-49 years,Neglected tropical diseases and malaria,Rate,2000,10.357415
8,Kenya,50-69 years,Neglected tropical diseases and malaria,Number,2000,905.006296
9,Kenya,50-69 years,Neglected tropical diseases and malaria,Rate,2000,45.708312


In [4]:
dat.cause = dat.cause.replace(['HIV/AIDS', 'Tuberculosis', 'Malaria'], ['hiv', 'tb', 'mal'])

dat = dat[dat.cause != 'Neglected tropical diseases and malaria']
dat.cause.value_counts()


tb     32980
mal    32980
hiv    32980
Name: cause, dtype: int64

In [5]:
#dat.merge(dat, left_on = ['location', 'age', 'metric', 'year', 'cause'], right_on= ['location', 'age', 'metric', 'year', 'cause']).head()

#dat is just the data frame
#'col' is what gets split into multiple new columns
#'samecols' is what to keep as the constants, they MUST be unique factor combinations for each row
#'vals' is what numbers we want to go into the new coulumns
#'rm_col' says whether to get rid of the original value column (only do if you don't want to use it again)
#Function to run the pivot
def splitrow(dat, col, samecols, vals, rm_col = True):
    df = copy.deepcopy(dat)
    #need to know how many unique levels there are for later
    n = len(df[col].value_counts())
    #Creating a single column to join on, as pivot requires it
    df['joincol'] = df[samecols].apply(lambda x: ''.join(x.astype(str)), axis=1)
    #Pivoting the table to make a new one with the columns we need
    tempdf = df.pivot(index = 'joincol', columns = col, values = vals)
    tempdf['joincol'] = tempdf.index
    #Merging together the two tables into one
    df = tempdf.merge(df, how = 'inner', on = 'joincol')
    #removing the join column
    df = df.drop(['joincol', vals], axis = 1)
    #If we want to remove the original column (no more joins to make), we remove duplicate rows
    if rm_col == True:
        df = df.drop(col, axis = 1)
        df = df.iloc[::n].reset_index()
        df = df.drop('index', axis = 1)
    return(df)

df = splitrow(dat, 'cause', ['location', 'age', 'metric', 'year'], 'val')

In [6]:
print(len(df))
df

32980


,hiv,mal,tb,location,age,metric,year
0,42.913628,771.122729,7138.658456,Afghanistan,15-49 years,Number,2000
1,45.543918,815.365474,7314.421455,Afghanistan,15-49 years,Number,2001
2,48.742938,894.241365,7277.983272,Afghanistan,15-49 years,Number,2002
3,52.410990,959.198303,7338.422563,Afghanistan,15-49 years,Number,2003
4,56.410928,973.515865,7419.208364,Afghanistan,15-49 years,Number,2004
5,60.669123,1005.361284,7380.569093,Afghanistan,15-49 years,Number,2005
6,64.718914,1030.273832,7327.581444,Afghanistan,15-49 years,Number,2006
7,68.885252,1036.418240,7180.659954,Afghanistan,15-49 years,Number,2007
8,73.398280,1048.697136,7042.798090,Afghanistan,15-49 years,Number,2008
9,77.863640,1054.140110,6897.832141,Afghanistan,15-49 years,Number,2009


In [7]:
df = splitrow(df, 'metric', ['location', 'age', 'year'], 'hiv', rm_col = False)
df = df.rename(columns = {'Number':'hiv_num', 'Rate':'hiv_rate'})

df = splitrow(df, 'metric', ['location', 'age', 'year'], 'tb', rm_col = False)
df = df.rename(columns = {'Number':'tb_num', 'Rate':'tb_rate'})

df = splitrow(df, 'metric', ['location', 'age', 'year'], 'mal')
df = df.rename(columns = {'Number':' mal_num', 'Rate':'mal_rate'})

df.head()

,mal_num,mal_rate,tb_num,tb_rate,hiv_num,hiv_rate,location,age,year
0,771.122729,9.054106,7138.658456,83.818267,42.913628,0.503869,Afghanistan,15-49 years,2000
1,815.365474,9.166599,7314.421455,82.231059,45.543918,0.512019,Afghanistan,15-49 years,2001
2,894.241365,9.564549,7277.983272,77.843223,48.742938,0.521340,Afghanistan,15-49 years,2002
3,959.198303,9.741092,7338.422563,74.524995,52.410990,0.532257,Afghanistan,15-49 years,2003
4,973.515865,9.412766,7419.208364,71.735113,56.410928,0.545428,Afghanistan,15-49 years,2004


In [8]:
codes = pd.read_csv('country_codes.csv')

codes = codes.drop_duplicates()
#codes.head()

In [9]:
#This merges in the country codes, but more importantly, it puts in info about what part of the world each country is in.
print(len(df))
df = df.merge(codes, how = 'left', left_on = 'location', right_on = 'location')
print(len(df))
df.columns

16490
16490


Index([' mal_num', 'mal_rate', 'tb_num', 'tb_rate', 'hiv_num', 'hiv_rate',
       'location', 'age', 'year', 'location_code', 'gbd_region',
       'gbd_superregion'],
      dtype='object')

Source of data
http://data.uis.unesco.org/#

In [171]:
demo = pd.read_csv('demographic_data.csv')
print(demo.Indicator.value_counts())
#demo.DEMO_IND.value_counts()

Total population                                                            3961
Population growth (annual %)                                                3879
Rural population (% of total population)                                    3865
Population aged 25-64 years                                                 3807
Population aged 65 years or older                                           3807
Population aged 15-24 years                                                 3807
Population aged 14 years or younger                                         3807
GDP (current LCU)                                                           3734
GDP per capita (current LCU)                                                3696
DEC alternative conversion factor (LCU per US$)                             3645
GNI (current LCU)                                                           3614
GDP (current US$)                                                           3603
GDP per capita (current US$)

200101               3961
SP_POP_GROW          3879
SP_RUR_TOTL_ZS       3865
200343               3807
200345               3807
200144               3807
200151               3807
NY_GDP_MKTP_CN       3734
NY_GDP_PCAP_CN       3696
PA_NUS_ATLS          3645
NY_GNP_MKTP_CN       3614
NY_GDP_MKTP_CD       3603
NY_GDP_PCAP_CD       3603
PA_NUS_FCRF          3590
NY_GDP_MKTP_KN       3574
NY_GDP_DEFL_ZS       3571
NY_GNP_PCAP_CN       3565
NY_GDP_MKTP_KD_ZG    3562
NY_GDP_MKTP_KD       3524
SP_DYN_IMRT_IN       3474
SP_DYN_TFRT_IN       3396
NY_GNP_PCAP_CD       3392
SP_DYN_LE00_IN       3389
PA_NUS_PPP           3382
NY_GDP_MKTP_PP_KD    3377
NY_GDP_PCAP_PP_KD    3377
NY_GDP_MKTP_PP_CD    3374
XTGOV_IMF            3372
PA_NUS_PPPC_RF       3349
NY_GNP_PCAP_PP_CD    3319
NY_GDP_PCAP_PP_CD    3151
PA_NUS_PRVT_PP       3127
SH_DYN_AIDS_ZS       2365
DT_TDS_DECT_GN_ZS    2146
SI_POV_DDAY          1030
SI_POV_2DAY           771
Name: DEMO_IND, dtype: int64

In [165]:
demo = pd.read_csv('demographic_data.csv')
#Only keeping 6 indicators, Total Population, Population Growth, Rural Population,
#   GDP (USD), GDP per capita (USD), and Fertility Rate.
demo = demo.drop(['TIME', 'Flag Codes', 'Flags'], axis = 1)
keepFacs = ['200101', 'SP_POP_GROW', 'SP_RUR_TOTL_ZS', 'SP_DYN_IMRT_IN',
            'NY_GDP_PCAP_CD', 'NY_GDP_MKTP_CD', 'SP_DYN_TFRT_IN', 'SP_DYN_LE00_IN']
keepInd = []
for i in range(len(demo)):
    if demo.DEMO_IND[i] not in keepFacs:
        keepInd.append(False)
    else:
        keepInd.append(True)
demo = demo[keepInd]

In [166]:
demo.head()

,DEMO_IND,Indicator,LOCATION,Country,Time,Value
0,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1999,1.755
1,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2000,1.756
2,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2001,1.739
3,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2002,1.756
4,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2003,1.748


In [167]:
#Reshaping the data here
dem_p = splitrow(demo, 'DEMO_IND', ['LOCATION', 'Country', 'Time'], 'Value')

dem_p = dem_p.rename(columns = {'200101':'Population', 'NY_GDP_MKTP_CD':'GDP', 'NY_GDP_PCAP_CD':'GDPperCapita',
                                'SP_DYN_IMRT_IN':'InfantMortality', 'SP_DYN_LE00_IN':'LifeExpectancy',
                               'SP_DYN_TFRT_IN':'FertilityRate', 'SP_POP_GROW':'Pop_growth', 'SP_RUR_TOTL_ZS':'RuralPop'})

#dem_p
dem_p


,Population,GDP,GDPperCapita,InfantMortality,LifeExpectancy,FertilityRate,Pop_growth,RuralPop,Indicator,LOCATION,Country,Time
0,89.005,1.722799e+09,19356.20339,NaN,73.67949,1.903,1.96056,52.873,"Fertility rate, total (births per woman)",ABW,Aruba,1999
1,90.853,1.873453e+09,20620.70063,NaN,73.72420,1.872,2.05503,53.283,"Life expectancy at birth, total (years)",ABW,Aruba,2000
2,92.898,1.920263e+09,20670.65566,NaN,73.78444,1.846,2.22593,53.661,Population growth (annual %),ABW,Aruba,2001
3,94.992,1.941095e+09,20434.29944,NaN,73.86217,1.823,2.22906,54.028,Rural population (% of total population),ABW,Aruba,2002
4,97.017,2.021302e+09,20834.51020,NaN,73.95883,1.803,2.10935,54.394,GDP (current US$),ABW,Aruba,2003
5,98.737,2.228279e+09,22567.82492,NaN,74.07190,1.787,1.75735,54.760,GDP per capita (current US$),ABW,Aruba,2004
6,100.031,2.331006e+09,23302.83199,NaN,74.19990,1.774,1.30204,55.125,Total population,ABW,Aruba,2005
7,101.220,2.623726e+09,25921.02605,NaN,74.48276,1.763,0.38406,55.853,"Fertility rate, total (births per woman)",ABW,Aruba,2007
8,101.353,2.791961e+09,27546.89939,NaN,74.62768,1.764,0.13131,56.217,"Life expectancy at birth, total (years)",ABW,Aruba,2008
9,101.453,2.498933e+09,24631.43486,NaN,74.77115,1.769,0.09862,56.579,Population growth (annual %),ABW,Aruba,2009


In [169]:
df_mod = df.merge(dem_p, how = 'left', left_on = ['location', 'year'], right_on = ['Country', 'Time'])

df_mod.columns

Index([' mal_num', 'mal_rate', 'tb_num', 'tb_rate', 'hiv_num', 'hiv_rate',
       'location', 'age', 'year', 'location_code', 'gbd_region',
       'gbd_superregion', 'Population', 'GDP', 'GDPperCapita',
       'InfantMortality', 'LifeExpectancy', 'FertilityRate', 'Pop_growth',
       'RuralPop', 'Indicator', 'LOCATION', 'Country', 'Time'],
      dtype='object')

In [351]:
test = df_mod['gbd_region'].astype('category')
test.unique()

[Asia, South, Europe, Central, North Africa/Middle East, NaN, Sub-Saharan Africa, Central, ..., Asia, East, Latin America, Central, Oceania, Europe, Western, Asia Pacific, high-income]
Length: 20
Categories (19, object): [Asia, South, Europe, Central, North Africa/Middle East, Sub-Saharan Africa, Central, ..., Latin America, Central, Oceania, Europe, Western, Asia Pacific, high-income]

In [271]:
#Fitting the model here, setting the predicted variable (mal_num) as being predicted by a combination of other factors
#mod = ols(formula = "hiv_rate ~ age + gbd_region + gbd_superregion + Population + GDP + GDPperCapita + FertilityRate + Pop_growth + RuralPop",
#          data = df_mod).fit()
formt = "hiv_rate ~ age + gbd_region + gbd_superregion + Population + GDP + GDPperCapita + FertilityRate + InfantMortality + Pop_growth + RuralPop"
#mod = ols(formula = formt, data = df_mod).fit()
#mod.summary()
mod = ols(formula = formt, data = df_mod)
model = mod.fit()
lassoFit = mod.fit_regularized(method='elastic_net', L1_wt=1.0, alpha = 0.01)
#Need to do this workaround because of bug in statsmodels
final = sm.regression.linear_model.OLSResults(mod, lassoFit.params, model.normalized_cov_params)
print(np.mean(final.resid**2))
final.summary()

#lassoFit.params

15154.818393615524


0.5447362405772872

In [354]:
#I know there's an easier way than this...
def makeRepSeq(n, f):
    s = n //  f
    l = []
    for i in range(1, f+1):
        l.extend([i] * s)
    return l

#Writing the funciton to run the cross validation
def runFoldCV(dat, params, y, folds):
    datC = dat.dropna(axis=0, how='any')
    dfRand = datC.sample(frac = 1).reset_index(drop = True)
    n = len(dfRand)
    splitSeq = np.array(makeRepSeq(n, folds))
    mseList = []
    form = y + '~' + ' + '.join(params)
    for i in range(1, folds+1):
        testdf = dfRand[splitSeq == i]
        traindf = dfRand[splitSeq != i]
        olsf = ols(formula = form, data = traindf)
        #modf = olsf.fit()
        modf = olsf.fit_regularized(method='elastic_net', L1_wt=1.0, alpha = 0.01)
        preds = modf.predict(testdf)
        mse = np.mean((preds - testdf[y])**2)
        mseList.append(mse)
        print(mse)
    return(np.mean(mseList))


terms = ['age', 'gbd_region', 'gbd_superregion', 'Population', 'GDP', 'GDPperCapita',
         'FertilityRate', 'InfantMortality', 'LifeExpectancy', 'Pop_growth', 'RuralPop']
yvar = 'mal_rate'
form = yvar + '~' + ' + '.join(terms)
runFoldCV(df_mod, terms, 'mal_rate', 5)

10560.812914451622
13202.504587559695
14171.987739732618
11094.435708596528
11964.014890014249


12198.751168070943

In [353]:
#Helper function to check type of data
def isQuant(L):
    for i in range(len(L)):
        if not (isinstance(L[i], float) or isinstance(L[i], int)):
            return False
    return True

def isCateg(L):
    for i in range(len(L)):
        #NOTE: Want better check, for when data is read in incorrectly.
        if not isinstance(L[i], str):
            return False
    return True

def getPredMatrix(dat, terms, y):
    predMatrix = pd.DataFrame()
    #modDat[y] = self.dat[y]
    predMatrix["Intercept"] = [1] * len(dat)
    for i in range(len(terms)):
        col = terms[i]
        print(col)
        var = dat[col]
        if dat.dtypes[col] == 'float64':
            predMatrix[col] = var
        else:
            #Splitting the different levels up based on how many there are.
            un_vals = var.unique()
            levels = sorted(list(set(var)))
            #Base Variable is the first alphabetically
            base = levels.pop(0)
            for l in levels:
                newCol = []
                #Need to manually go through and code in 0s and 1s
                for obs in var:
                    if obs == l:
                        newCol.append(1)
                    else:
                        newCol.append(0)
                colName = col + "[T." + l + "]"
                predMatrix[colName] = newCol
    predMatrix[y] = dat[y]
    return predMatrix

terms = ['age', 'gbd_region', 'gbd_superregion', 'Population', 'GDP', 'GDPperCapita',
         'FertilityRate', 'InfantMortality', 'LifeExpectancy', 'Pop_growth', 'RuralPop']
test = df_mod.dropna(axis=0, how='any')
X = test
X = getPredMatrix(X, terms, 'hiv_rate')
X
#X = X.dropna(axis=0, how='any').reset_index(drop = True)
#Y = X['hiv_rate']
#X = X.drop('hiv_rate', axis = 1)

#reg_mod = LinearRegression()
#reg_mod.fit(X, Y)
#type(df_mod['gbd_region'][1])
#list(set(df_mod.dropna(axis=0, how='any')['gbd_region']))
#for idx, col_name in enumerate(X.columns):
#    print("The coefficient for {} is {}".format(col_name, reg_mod.coef_[idx]))
#reg_mod.score(X, Y)

age
gbd_region
gbd_superregion
Population
GDP
GDPperCapita
FertilityRate
InfantMortality
LifeExpectancy
Pop_growth
RuralPop


,Intercept,age[T.5-14 years],age[T.50-69 years],age[T.70+ years],age[T.Under 5],"gbd_region[T.Asia, East]","gbd_region[T.Asia, South]","gbd_region[T.Asia, Southeast]",gbd_region[T.Caribbean],"gbd_region[T.Europe, Central]",...,gbd_superregion[T.Sub-Saharan Africa],Population,GDP,GDPperCapita,FertilityRate,InfantMortality,LifeExpectancy,Pop_growth,RuralPop,hiv_rate
0,1,0,0,0,0,0,1,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,0,0,0,1,0,0,0,...,0,20966.463,2.461666e+09,117.40969,7.392,88.4,56.06071,4.25150,78.404,0.512019
2,1,0,0,0,0,0,1,0,0,0,...,0,21979.923,4.128821e+09,187.84509,7.271,86.1,56.65354,4.72053,78.085,0.521340
3,1,0,0,0,0,0,1,0,0,0,...,0,23064.851,4.583644e+09,198.72854,7.136,83.6,57.26588,4.81804,77.763,0.532257
4,1,0,0,0,0,0,1,0,0,0,...,0,24118.979,5.285466e+09,219.14135,6.988,81.1,57.89122,4.46892,77.438,0.545428
5,1,0,0,0,0,0,1,0,0,0,...,0,25070.798,6.275074e+09,250.29413,6.827,78.5,58.51559,3.87047,77.105,0.561319
6,1,0,0,0,0,0,1,0,0,0,...,0,25893.450,7.057598e+09,272.56308,6.651,76.0,59.12595,3.22863,76.763,0.581775
7,1,0,0,0,0,0,1,0,0,0,...,0,26616.792,9.843842e+09,369.83580,6.460,73.4,59.70885,2.75523,76.413,0.604157
8,1,0,0,0,0,0,1,0,0,0,...,0,27294.031,1.019053e+10,373.36112,6.254,70.8,60.25673,2.51257,76.054,0.628862
9,1,0,0,0,0,0,1,0,0,0,...,0,28004.331,1.248694e+10,445.89330,6.038,68.2,60.76461,2.56911,75.687,0.650120


In [183]:
newterms = copy.copy(terms)
terms = ['age', 'gbd_region', 'gbd_superregion', 'Population', 'GDP', 'GDPperCapita',
         'FertilityRate', 'InfantMortality', 'LifeExpectancy', 'Pop_growth', 'RuralPop']

X = df_mod[terms]
Y = df_mod['mal_rate']
reg_mod = LinearRegression()
reg_mod.fit(X, Y)


ValueError: could not convert string to float: 'Sub-Saharan Africa'

In [145]:
np.array([1,2,3,1]) == 1
X[np.array([1,2,3,1]) != 1]
' + '.join(['1','32', 'safsd', '  ', 'a90843'])

'1 + 32 + safsd +    + a90843'